# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import matplotlib.pyplot as plt
import simfin as sf



In [2]:
# import of functions from custom modules
from get_prices import download_ticker_prices
from get_fundamentals import *
from get_peers import download_peer_data, get_peer_data_from_disk
from get_company_info import load_company_info_from_disk
from analysis_utils import create_pandas_profiling_report, list_all_directory_files

## Step 1: Scope the Project and Gather Data

### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

#### 1.1 Stock information
- Ticker symbol list: "http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt"
    - Symbol Look-Up/Directory Data Fields & Definitions: http://www.nasdaqtrader.com/trader.aspx?id=symboldirdefs
- Company information: Kaggle dataset from 2019 https://www.kaggle.com/marketahead/all-us-stocks-tickers-company-info-logos 
- Peer group information: IEX Cloud API https://iexcloud.io/docs/api/#peer-groups

##### 1.1.1 Ticker Symbol List

In [3]:
# get ticket symbol list
symbol_df = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')

In [4]:
symbol_df.head(2)

,Nasdaq Traded,Symbol,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
0,Y,A,"Agilent Technologies, Inc. Common Stock",N,,N,100.0,N,NaN,A,A,N
1,Y,AA,Alcoa Corporation Common Stock,N,,N,100.0,N,NaN,AA,AA,N


In [4]:
# exclude test issues
symbol_df = symbol_df[(symbol_df['Test Issue'] == 'N')]
# exclude companies that are bankrupt
symbol_df = symbol_df[symbol_df['Financial Status'].isna() | (symbol_df['Financial Status']=='N')]
# exclude ETFs
symbol_df = symbol_df[symbol_df['ETF']=='N']
symbol_list = symbol_df['NASDAQ Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbol_list)))

total number of symbols traded = 7960


##### 1.1.2 Company Info

In [6]:
company_info_df = load_company_info_from_disk(symbol_list)

In [12]:
company_info_df.head(1)

,ticker,company name,short name,industry,description,website,logo,ceo,exchange,market cap,sector,tag 1,tag 2,tag 3
0,A,Agilent Technologies Inc.,Agilent,Medical Diagnostics & Research,Agilent Technologies Inc is engaged in life sc...,http://www.agilent.com,A.png,Michael R. McMullen,New York Stock Exchange,2.421807e+10,Healthcare,Healthcare,Diagnostics & Research,Medical Diagnostics & Research


In [8]:
create_pandas_profiling_report(company_info_df, 'company_info_df')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 250.11it/s]

Pandas profiling report of file company_info_df created



In [16]:
ticker_count = company_info_df['ticker'].nunique()
print(f'Number of companies in company_info_df: {ticker_count}')
# reduce symbol_list to those where company information is available
symbol_list = company_info_df['ticker'].unique().tolist()

Number of companies in company_info_df: 4559


##### 1.1.3 Peer group information


In [17]:
# initial download of peer data from API
#peer_df = download_peer_data(symbol_list)

In [18]:
peer_df.shape

(4559, 3)

In [10]:
# load downloaded peer data from disk
peer_df = get_peer_data_from_disk(symbol_list)

#### 1.2 Fundamental indicators 
- Return on Investment Capital (ROIC)
    - Source: Income Statement
        - net income: n_i -> column 'Net Income'
        - dividend: di -> column 'Dividends Paid'
        - debt: de -> summed up with equity in column 'Total Liabilities & Equity'
        - equity: eq
    - Definition: ROIC = (ni - di) / (de + eq)
- Sales Growth Rate
    - Source: Income Statement
    - Definition:  Sales is equal to column 'revenue'
- Earnings per Share Growth Rate
    - Source: Income Statement
        - Earnings: ea -> column 'Net Income'
        - number of shares : sh -> column 'Shares (Basic)'
    - Definition: ea / sh
- Book Value per Share Growth Rate
    - Source: Balance Sheet
        - Total Equity: t_e -> column 'Total Equity'
        - Prefered Equity: p_e (not available)
        - number of shares : sh -> column 'Shares (Basic)'
    - Definition: (t_e - p_e) / sh
- Free Cash Flow Growth Rate
    - Source: 
        - Cash Flow: 
            - Cashflow from Operating Activities: cf_oa -> column 'Net Cash from Operating Activities'
            - Capital Expenditure: capex -> column 'Net Cash from Investing Activities'
        - Income Statement:
            - Interest Expenses: i_e -> exclude for simplicity, column 'Interest Expense, Net'
            - Tax shield on Interest Expense: t_i_e  -> exclude for simplicity
    - Definition: Free Cashflow (f_cf) = cf_oa + i_e - t_i_e - capex

In [9]:
# initialize simfin API
init_simfin_api()

In [10]:
market='us'
variant='annual'
# download cashflow data from the SimFin server and load into a Pandas DataFrame.
cashflow_df = sf.load_cashflow(variant=variant, market=market)
cashflow_df = cashflow_df.reset_index()
# Download the data from the SimFin server and load into a Pandas DataFrame.
income_sm_df = sf.load_income(variant=variant, market=market)
income_sm_df = income_sm_df.reset_index()
# Download the data from the SimFin server and load into a Pandas DataFrame.
balance_st_df = sf.load_balance(variant=variant, market=market)
balance_st_df = balance_st_df.reset_index()
# create Pandas Profiling Report for each DataFrame
for df, df_name in zip([cashflow_df, income_sm_df, balance_st_df],
                        ['cashflow_df', 'income_sm_df', 'balance_st_df']):
    create_pandas_profiling_report(df, df_name)

Dataset "us-cashflow-annual" on disk (4 days old).
- Loading from disk ... Done!
Dataset "us-income-annual" on disk (4 days old).
- Loading from disk ... Done!
Dataset "us-balance-annual" on disk (2 days old).
Summarize dataset:   3%|▎         | 1/37 [00:00<00:03,  9.72it/s, Describe variable:Report Date]

Pandas profiling report of file cashflow_df created
Summarize dataset:   0%|          | 0/39 [00:00<?, ?it/s, Describe variable:Currency]

Pandas profiling report of file income_sm_df created
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 80.98it/s]

Pandas profiling report of file balance_st_df created



In [3]:
# combine data
symbol_list = ['AAPL', 'GOOG']
period_dict = {'start_date':2010,
                'end_date':2019}

# test combine fundamentals and calculate top5 kpis
test_fundamental_df = combine_fundamentals(symbol_list, period_dict)
test_fundamental_df = calculate_top5_kpi(test_fundamental_df)

# test calculate growth kpi df
test_growth_df = calculate_growth_rates(test_fundamental_df, agg_func='mean')
test_growth_df.head()

Dataset "us-cashflow-annual" on disk (9 days old).
- Loading from disk ... Done!
Dataset "us-income-annual" on disk (9 days old).
- Loading from disk ... Done!
Dataset "us-balance-annual" on disk (6 days old).
- Loading from disk ... Done!
Combined all fundamental data from financial statements to one Dataframe.
Calculated roic and added it to Dataframe
Calculated eps and added it to Dataframe
Calculated bvps and added it to Dataframe
Calculated fcf and added it to Dataframe
top5 KPIs added to fundamental data
Calculated KPI growth from year to year.
Calculated 5 and 10 year growth rate


,Ticker,revenue_gr_2019,eps_2019,roic_gr_5yr,revenue_gr_5yr,eps_gr_5yr,bvps_gr_5yr,fcf_gr_5yr,yrs_in_5yr,roic_gr_10yr,revenue_gr_10yr,eps_gr_10yr,bvps_gr_10yr,fcf_gr_10yr,yrs_in_10yr
0,AAPL,-0.020411,2.991446,0.02,0.08,0.15,0.02,-0.12,5,-0.02,0.19,0.24,0.14,0.10,10
1,GOOG,0.183001,49.585906,0.12,0.20,0.30,0.14,0.14,5,0.04,0.21,0.16,0.10,0.18,10


#### 1.3 Pricing information ####
The purpose of this data is to evaluate the so-called sticker prices and margin of safety based on current stock prices. It can also be used for backtesting criteria on historic data.
- Sticker price calculation
    - future period fp, by default 10 years
    - Sticker price = future market price / (1 + exp_rr)^fp
    - expected annual return rate exp_rr, by default 15%
    - future market price = future P/E * estimated future EPS
        - future P/E = min(pe_default, pe_5yr_avg)
            - default price per earnings pe_default: 2* rule #1 growth rate (see below)
            - 5 year average of annual price per earnings pe_5yr_avg
                - annual price per earnings pe = price/ eps
                    - annual price = median of daily low prices in month December
        - estimated future EPS = current EPS * (1+ rule1_gr)^fp
            - rule #1 growth rate rule1_gr = min(bvps_gr_5yr, eps_gr_5yr)

- Margin of safety: half the the sticker price.

In [ ]:
# test_fundamental_df
# test_growth_df

In [3]:
# adapt yfinance notebook to python script that downloads ticker symbols
symbol_list = ['AAPL']

download_ticker_prices(symbol_list)

Total number of valid symbols downloaded = 1


## Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





## Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model
- Dimensional tables
    - company information, including peers
- Fact tables:
    - fundamental indicators
    - price history

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.